In [1]:
using UnfoldSim
using Random
using DataFrames
using CairoMakie
using StableRNGs
using UnfoldMakie
using Plots
using StatsBase

In [7]:
print("hi julia")

hi julia

In [11]:
#check single subject
noiselevel = 0.2

rngSS = StableRNG(1)

data_SS, events_SS = UnfoldSim.predef_eeg(
    rngSS;
    n_repeats = 100,  
    multichannel = true,  
    noiselevel = noiselevel,
    event_order_function = shuffle,
    return_epoched = true       # returns (timepoints, channels, trials)
)

println(size(data_SS))
println(first(events_SS, 10))


Please cite: HArtMuT: Harmening Nils, Klug Marius, Gramann Klaus and Miklody Daniel - 10.1088/1741-2552/aca8ce
(227, 44, 2000)
10×3 DataFrame
 Row │ continuous  condition  latency 
     │ Float64     String     Int64   
─────┼────────────────────────────────
   1 │  -0.555556  face            70
   2 │   3.88889   car            138
   3 │   2.77778   car            208
   4 │  -2.77778   car            271
   5 │  -3.88889   car            329
   6 │  -5.0       face           379
   7 │   0.555556  car            447
   8 │   0.555556  face           514
   9 │  -1.66667   car            580
  10 │   3.88889   face           635


In [5]:
#check single subject
noiselevel2 = 6

myrngSS = StableRNG(1)

data_noise, events_noise = UnfoldSim.predef_eeg(
    myrngSS;
    n_repeats = 100,  
    multichannel = true,  
    noiselevel = noiselevel2,
    return_epoched = true       # returns (timepoints, channels, trials)
)

println(size(data_noise))
println(first(events_noise, 10))

Please cite: HArtMuT: Harmening Nils, Klug Marius, Gramann Klaus and Miklody Daniel - 10.1088/1741-2552/aca8ce
(227, 44, 2000)
10×3 DataFrame
 Row │ continuous  condition  latency 
     │ Float64     String     Int64   
─────┼────────────────────────────────
   1 │  -0.555556  face            70
   2 │   3.88889   car            138
   3 │   2.77778   car            208
   4 │  -2.77778   car            271
   5 │  -3.88889   car            329
   6 │  -5.0       face           379
   7 │   0.555556  car            447
   8 │   0.555556  face           514
   9 │  -1.66667   car            580
  10 │   3.88889   face           635


In [11]:
#extract 32 channels from 227 channels
using UnfoldSim

# Assume l32 is your list of 32 target channel labels
 l32, _ = UnfoldMakie.example_montage("biosemi_32") 

# 1. Get the full list of labels used in the simulation (227 channels)
hart = UnfoldSim.headmodel()
full_labels_227 = hart.electrodes["label"]
# 2. Find the indices in the 227-channel list that match your 64 target labels
indices_to_extract = findall(l -> l in l32, full_labels_227)
data_noise_32 = data_noise[indices_to_extract, :, :]

# Output should be: (32, Time, Trials, Subjects)
ordered_indices = [findfirst(isequal(ch), full_labels_227) for ch in l32]  #check where label occurs in full_labels_227
data_32_ch = data_noise[ordered_indices, :, :]
# Verifyorder
extracted_labels_ordered = full_labels_227[ordered_indices]
#println("Channels after reordering:")
println(extracted_labels_ordered)
#println("Channels in l32")
println(l32)
println(size(data_noise_32))
#Confirm order matches target
#println("Order matches l32? ", extracted_labels_ordered == l32)
println("Original data dimensions data_32_ch: $(size(data_noise))")
println("Extracted 32-channel dimensions: $(size(data_noise_32))") 


Please cite: HArtMuT: Harmening Nils, Klug Marius, Gramann Klaus and Miklody Daniel - 10.1088/1741-2552/aca8ce
["Fp1", "AF3", "F7", "F3", "FC1", "FC5", "T7", "C3", "CP1", "CP5", "P7", "P3", "Pz", "PO3", "O1", "Oz", "O2", "PO4", "P4", "P8", "CP6", "CP2", "C4", "T8", "FC6", "FC2", "F4", "F8", "AF4", "Fp2", "Fz", "Cz"]
["Fp1", "AF3", "F7", "F3", "FC1", "FC5", "T7", "C3", "CP1", "CP5", "P7", "P3", "Pz", "PO3", "O1", "Oz", "O2", "PO4", "P4", "P8", "CP6", "CP2", "C4", "T8", "FC6", "FC2", "F4", "F8", "AF4", "Fp2", "Fz", "Cz"]
(32, 44, 2000)
Original data dimensions data_32_ch: (227, 44, 2000)
Extracted 32-channel dimensions: (32, 44, 2000)


# 1. Avg across trials - Single Subject Design 44x32 

In [13]:
using CSV, DataFrames, Statistics
# Average over trials (dimension 3)
erp_data = mean(data_noise_32, dims=3)  # shape (channels, timepoints, 1)
erp_data = dropdims(erp_data; dims=3)  # shape (channels, timepoints)
sampling_rate = 100
dt_ms = 1000 / sampling_rate
timepoints = size(erp_data, 2)
time_ms = (0:timepoints-1) .* dt_ms

# transpose to (timepoints, channels)
erp_data_t = permutedims(erp_data, (2, 1))
channel_names = ["ch$(i)" for i in 1:size(erp_data,1)]
df = DataFrame(time_ms = time_ms)
for (i, ch_name) in enumerate(channel_names)
    df[!, ch_name] = erp_data_t[:, i]
end
#CSV.write("21_ERPAvg_SingleSubject_15dec2.csv", df)
CSV.write("erpWithNoise.csv", df)

println("ERP CSV written with size: ", size(df))

using CSV, DataFrames, Statistics
# Average over trials (dimension 3)
erp_data = mean(data_noise_32, dims=3)  # shape (channels, timepoints, 1)
erp_data = dropdims(erp_data; dims=3)  # shape (channels, timepoints)
sampling_rate = 100
dt_ms = 1000 / sampling_rate
timepoints = size(erp_data, 2)
time_ms = (0:timepoints-1) .* dt_ms

ERP CSV written with size: (44, 33)


0.0:10.0:430.0

In [29]:
size(data_noise_32)

(32, 44, 2000)

In [ ]:
# dont average trials because tphate is suppose to denoise it

In [35]:
using DataFrames, CSV, Statistics
# Correcting the dimensions based on your specific input: (32, 44, 2000)
# n_ch = 32, n_time = 44, n_trials = 2000
n_ch, n_time, n_trials = size(data_noise_32)

# 1. Permute to (Time, Trials, Channels)
# To get the trials to follow each other in a continuous time-series:
# Move dimension 2 (Time) to position 1
# Move dimension 3 (Trials) to position 2
# Move dimension 1 (Channels) to position 3
data_permuted = permutedims(data_noise_32, (2, 3, 1)) 

# 2. Reshape to (88000, 32)
# The ':' flattens (44 * 2000) into 88,000 rows.
# The second argument ensures we have n_ch (32) columns.
data_continuous = reshape(data_permuted, :, n_ch)

# 3. Create DataFrame with proper column naming (32 columns)
df_continuous = DataFrame(data_continuous, [Symbol("ch$i") for i in 1:n_ch])

# 4. Save and Verify
CSV.write("Try1-Continuous-NoisyData.csv", df_continuous)
println("Expected size: (88000, 32)")
println("Actual size of df is $(size(df_continuous))")

Expected size: (88000, 32)
Actual size of df is (88000, 32)


In [37]:
using DataFrames, CSV

# Data dimensions: (Channels: 32, Time: 44, Trials: 2000)
n_ch, n_time, n_trials = size(data_noise_32)

# 1. Permute to (Time, Trials, Channels)
# This stacks all timepoints first, then all trials, then channels.
data_permuted = permutedims(data_noise_32, (2, 3, 1)) 

# every ROW is a single timepoint in a specific trial.
data_rows = reshape(data_permuted, :, n_ch)

# 3. Create a DataFrame
df = DataFrame(data_rows, [Symbol("ch$i") for i in 1:n_ch])

# 4. (Optional) Add Metadata columns for easier plotting later
# This helps you color-code your T-PHATE plots by time or trial
df[!, :time_idx] = repeat(1:n_time, n_trials)
df[!, :trial_id] = repeat(1:n_trials, inner=n_time)

CSV.write("Try2.csv", df)
println("Final Matrix for T-PHATE: $(size(df)) (Rows=Timepoints, Cols=Channels)")

Final Matrix for T-PHATE: (88000, 34) (Rows=Timepoints, Cols=Channels)


# 2. Average across trials per condition - Single Subject Design


In [43]:
using Statistics, DataFrames, CSV

# 1. Identify indices
f_idx = findall(x -> x == "face", events_SS.condition)
c_idx = findall(x -> x == "car", events_SS.condition)

# 2. Average trials (32 channels x 44 timepoints)
face_avg = dropdims(mean(data_32_ch[:, :, f_idx], dims=3), dims=3)
car_avg = dropdims(mean(data_32_ch[:, :, c_idx], dims=3), dims=3)

# 3. Create Metadata first to ensure they are the first columns
df_metadata = DataFrame(
    Time_ms = repeat(collect(0:10:430), outer=2),
    Condition = repeat(["Face", "Car"], inner=44)
)

# 4. Create Channel data (Transpose averages to get 44 rows per condition)
df_channels = vcat(
    DataFrame(face_avg', [Symbol("Ch$i") for i in 1:32]),
    DataFrame(car_avg', [Symbol("Ch$i") for i in 1:32])
)

# 5. Horizontal Concatenation puts metadata on the left
df_cond = hcat(df_metadata, df_channels)

CSV.write("Case2-SS-Condition-Averages.csv", df_cond)
println("Size of df is $(size(df_cond))")
println("Columns: $(names(df_cond)[1:5])...")

Size of df is (88, 34)
Columns: ["Time_ms", "Condition", "Ch1", "Ch2", "Ch3"]...


# 3. Average All trials, all conditions—single subject design


In [45]:
using DataFrames, CSV

# 1. Reshape the 3D array (32, 44, 2000) into a 2D matrix (88000, 32)
# We permute dimensions to get (Time*Trials, Channels)
data_reshaped = reshape(permutedims(data_32_ch, (2, 3, 1)), n_time * n_trials, n_ch)

# 2. Create metadata columns using repeats and tiles
time_vector = repeat(collect(0:10:430), n_trials)
trial_id_vector = repeat(1:n_trials, inner = n_time)
condition_vector = repeat(events_SS.condition, inner = n_time)

# 3. Construct the DataFrame in one shot with the correct order
final_df = DataFrame(
    time_ms = time_vector,
    trial_id = trial_id_vector,
    condition = condition_vector
)
# 4. Add the channel columns
for i in 1:n_ch
    final_df[!, Symbol("Ch$i")] = data_reshaped[:, i]
end

CSV.write("Case3-SS-All-Trials.csv", final_df)
println("size of df is $(size(final_df))")

size of df is (88000, 35)


In [ ]:
# MS DESIGN

In [49]:
n_subjects = 20
items = 20  #multiple of 4 because of continous conditions and it has 2 conditions 10 levels. 
noiselevel = 0.2

rng = StableRNG(1)

data_MS, events_MS = UnfoldSim.predef_eeg(
    rng,
    n_subjects;
    n_items = items,       
multichannel = true,  # - Multichannel: If set to `true` uses: `["Right Occipital Pole", "Left Postcentral Gyrus", "Left Superior Frontal Gyrus",]`.
    noiselevel = noiselevel,
    return_epoched = true       # returns (timepoints, channels, subjects, trials)
)

println(size(data_MS))
println(first(events_MS, 40))


Please cite: HArtMuT: Harmening Nils, Klug Marius, Gramann Klaus and Miklody Daniel - 10.1088/1741-2552/aca8ce
(227, 44, 20, 20)
40×5 DataFrame
 Row │ subject  item    continuous  condition  latency 
     │ String   String  Float64     String     Int64   
─────┼─────────────────────────────────────────────────
   1 │ S01      I01      -5.0       car             70
   2 │ S01      I02      -3.88889   car            138
   3 │ S01      I13      -2.77778   face           208
   4 │ S01      I04      -1.66667   car            271
   5 │ S01      I17       1.66667   face           329
   6 │ S01      I18       2.77778   face           379
   7 │ S01      I16       0.555556  face           447
   8 │ S01      I19       3.88889   face           514
   9 │ S01      I06       0.555556  car            580
  10 │ S01      I15      -0.555556  face           635
  11 │ S01      I12      -3.88889   face           697
  12 │ S01      I09       3.88889   car            763
  13 │ S01      I11      -5.

In [51]:
#extract 32 channels from 227 channels
# Assume l32 is your list of 32 target channel labels
# 2. Find the indices in the 227-channel list that match your 64 target labels
data_32_ch_MS = data_MS[indices_to_extract, :, :,:]
println("Original data dimensions data_32_ch_MS: $(size(data_MS))")
println("Extracted 32-channel dimensions: $(size(data_32_ch_MS))") 
# Output should be: (32, Time, Trials, Subjects)
ordered_indicesMS = [findfirst(isequal(ch), full_labels_227) for ch in l32]  #check where label occurs in full_labels_227
data_32_ch_MS = data_MS[ordered_indicesMS, :, :,:]
# Verify order
extracted_labels_orderedMS = full_labels_227[ordered_indicesMS]
#println("Channels after reordering:")
println(extracted_labels_orderedMS)
#println("Channels in l32")
println(l32)
println(size(data_32_ch_MS))
#Confirm order matches target
#println("Order matches l32? ", extracted_labels_ordered == l32)
println("Data size of single subject data_32_ch_MS  is $(size(data_32_ch_MS))")
println("Size of single subject events is $(size(events_MS))")  #(20 x 20 x 5 unique conditions)


Original data dimensions data_32_ch_MS: (227, 44, 20, 20)
Extracted 32-channel dimensions: (32, 44, 20, 20)
["Fp1", "AF3", "F7", "F3", "FC1", "FC5", "T7", "C3", "CP1", "CP5", "P7", "P3", "Pz", "PO3", "O1", "Oz", "O2", "PO4", "P4", "P8", "CP6", "CP2", "C4", "T8", "FC6", "FC2", "F4", "F8", "AF4", "Fp2", "Fz", "Cz"]
["Fp1", "AF3", "F7", "F3", "FC1", "FC5", "T7", "C3", "CP1", "CP5", "P7", "P3", "Pz", "PO3", "O1", "Oz", "O2", "PO4", "P4", "P8", "CP6", "CP2", "C4", "T8", "FC6", "FC2", "F4", "F8", "AF4", "Fp2", "Fz", "Cz"]
(32, 44, 20, 20)
Data size of single subject data_32_ch_MS  is (32, 44, 20, 20)
Size of single subject events is (400, 5)


# 4. Average across trials - Multi Subject Design

In [87]:
# data_32_ch_MS: (32 Channels, 44 timepoints, 20 trials, 20 subjects)
# 1. Average across the 20 trials per subject
sub_averages = dropdims(mean(data_32_ch_MS, dims=3), dims=3) # Result: (32, 44, 20)

df_multi = DataFrame()

for s in 1:20
    sub_slice = sub_averages[:, :, s]'
    tmp = DataFrame(sub_slice, [Symbol("Ch$i") for i in 1:32])
    tmp[!, :Time_ms] = collect(0:10:430)
    tmp[!, :SubjectID] .= s
    append!(df_multi, tmp)
end

CSV.write("Case4-MS-AvgTrials.csv", df_multi)
println("size of df is $(size(df_multi))")


size of df is (880, 34)


# 5. Average across trials per condition - Multi Subject Design


In [89]:
using Statistics, DataFrames, CSV

# 1. Identify dimensions correctly
n_channels = size(data_32_ch_MS, 1) # 32
time_len   = size(data_32_ch_MS, 2) # 44
n_subjects = size(data_32_ch_MS, 4) # 20

# 2. Local indices for the 20 trials
events_local = events_MS[1:20, :]
f_idx = findall(x -> x == "face", events_local.condition)
c_idx = findall(x -> x == "car", events_local.condition)

all_subject_data = []

for s in 1:n_subjects
    # Average trials: Result is (32, 44)
    face_avg = dropdims(mean(data_32_ch_MS[:, :, f_idx, s], dims=3), dims=3)
    car_avg  = dropdims(mean(data_32_ch_MS[:, :, c_idx, s], dims=3), dims=3)
    
    # 3. Build Metadata (88 rows total)
    meta = DataFrame(
        SubjectID = fill(s, time_len * 2),
        Condition = repeat(["Face", "Car"], inner=time_len),
        Time_ms   = repeat(collect(0:10:(10*(time_len-1))), 2)
    )
    
    # 4. Build Channel columns - ADD THE TRANSPOSE ' HERE
    # Transposing changes (32, 44) into (44, 32) so rows = time and cols = channels
    ch_names = [Symbol("Ch$i") for i in 1:n_channels]
    
    ch_data = vcat(
        DataFrame(face_avg', ch_names), 
        DataFrame(car_avg',  ch_names)
    )
    
    push!(all_subject_data, hcat(meta, ch_data))
end

# Combine everything
df_ms_final = vcat(all_subject_data...)

CSV.write("Case5_MS_Final_Fixed.csv", df_ms_final)
println("Success! Rows: $(nrow(df_ms_final)), Cols: $(ncol(df_ms_final))")

Success! Rows: 1760, Cols: 35


In [ ]:
# Case 6: Keep 3 subjects to compare Multi Subject Condition Avg across trials

In [142]:
using Statistics, DataFrames, CSV

# 2. Local indices for the 20 trials
events_local = events_MS[1:20, :]
f_idx = findall(x -> x == "face", events_local.condition)
c_idx = findall(x -> x == "car", events_local.condition)

all_subject_data = []

for s in 1:2  
    # Average trials: Result is (32, 44)
    face_avg = dropdims(mean(data_32_ch_MS[:, :, f_idx, s], dims=3), dims=3)
    car_avg  = dropdims(mean(data_32_ch_MS[:, :, c_idx, s], dims=3), dims=3)
    
    # 3. Build Metadata (88 rows total)
    meta = DataFrame(
        SubjectID = fill(s, time_len * 2),
        Condition = repeat(["Face", "Car"], inner=time_len),
        Time_ms   = repeat(collect(0:10:(10*(time_len-1))), 2)
    )
    
    # Transposing changes (32, 44) into (44, 32) so rows = time and cols = channels
    ch_names = [Symbol("Ch$i") for i in 1:n_channels]
    
    ch_data = vcat(
        DataFrame(face_avg', ch_names), 
        DataFrame(car_avg',  ch_names)
    )
    
    push!(all_subject_data, hcat(meta, ch_data))
end

# Combine everything
df_ms_final = vcat(all_subject_data...)

CSV.write("Case6_2subjects_conditions.csv", df_ms_final)
println("Success! Rows: $(nrow(df_ms_final)), Cols: $(ncol(df_ms_final))")

Success! Rows: 176, Cols: 35


In [75]:
time_len = size(data_32_ch_MS, 2)    # Should be 44


44

In [83]:
n_channels = size(data_32_ch_MS, 1)  # Should be 32


32

In [ ]:
n_subjects = size(data_32_ch_MS, 4)  # 20

In [93]:
#=n_subjects = 20
items2 =40  #multiple of 4 because of continous conditions and it has 2 conditions 10 levels. 
noiselevel = 0.2

rng = StableRNG(1)

data_MSSS, events_MSSSS = UnfoldSim.predef_eeg(
    rng,
    n_subjects;
    n_items = items2,       
multichannel = true,  # - Multichannel: If set to `true` uses: `["Right Occipital Pole", "Left Postcentral Gyrus", "Left Superior Frontal Gyrus",]`.
    noiselevel = noiselevel,
    return_epoched = true       # returns (timepoints, channels, subjects, trials)
)

println(size(data_MSSS))

Please cite: HArtMuT: Harmening Nils, Klug Marius, Gramann Klaus and Miklody Daniel - 10.1088/1741-2552/aca8ce
(227, 44, 40, 20)
